In [2]:
# imports
import requests
import pandas as pd
import os

In [3]:
api_key = os.getenv("fsq_api_key")

In [4]:
# Function to make an http request to foursqure for each bike station
def get_poi_fs(latitude, longitude, radius, api_key, categories):    
    url = f'https://api.foursquare.com/v3/places/search?ll={latitude},{longitude}&radius={radius}'

    # Create dictionary for headers
    headers = {"Accept": "application/json"}
    headers['Authorization'] = api_key
    reponse = requests.get(url, headers=headers)
    reponse_data = reponse.json()
    
    return reponse_data['results']

In [5]:
# #Create list with desired categories from Yelp
# dining_cat_ids = ['restaurants', 'bars', 'pubs']

In [6]:
# assuming api_key is a variable containing the API key
res = get_poi_fs(41.3874, 2.1686, 1000, api_key, categories=['restaurants', 'bars', 'pubs'])


In [7]:
# to explore the result of a single POI from the testing call
res[0]

{'fsq_id': '5c4714533ba767002caedd2b',
 'categories': [{'id': 13302,
   'name': 'Mediterranean Restaurant',
   'short_name': 'Mediterranean',
   'plural_name': 'Mediterranean Restaurants',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mediterranean_',
    'suffix': '.png'}}],
 'chains': [],
 'closed_bucket': 'VeryLikelyOpen',
 'distance': 47,
 'geocodes': {'drop_off': {'latitude': 41.387677, 'longitude': 2.168101},
  'main': {'latitude': 41.387526, 'longitude': 2.167989},
  'roof': {'latitude': 41.387526, 'longitude': 2.167989}},
 'link': '/v3/places/5c4714533ba767002caedd2b',
 'location': {'address': 'Rambla Catalunya, 3',
  'admin_region': 'Cataluña',
  'country': 'ES',
  'cross_street': '',
  'formatted_address': 'Rambla Catalunya, 3, 08007 Barcelona Catalunya',
  'locality': 'Barcelona',
  'postcode': '08007',
  'region': 'Catalunya'},
 'name': 'Honest Greens Rambla Catalunya',
 'related_places': {},
 'timezone': 'Europe/Madrid'}

Put your parsed results into a DataFrame ___

In [8]:
# load the df of the bike stations containing the latitude and longitude
bike_stations_load1 = pd.read_csv('../data/bike_stations_barcelona.csv', index_col=0)

In [9]:
# Create an empty dataframe
foursquare_df = pd.DataFrame()

In [10]:
# Loop through the bike_stations df to extract the lat/long/id of each row and make a new http request
for index, row in bike_stations_load1.iterrows():
    # Make the http request using row attributes
    poi_fs = get_poi_fs(row['latitude'], row['longitude'], 1000, api_key, categories=None)
    # Turn the response into a dataframe
    new_df = pd.json_normalize(poi_fs, record_path=['categories'], meta=['fsq_id', 'name', 'distance', ['location', 'address']], meta_prefix='poi_', errors='ignore')
    # Add a column with the corresponding bike station id
    new_df['station_id'] = row['id'] 
    # Add the new dataframe to yelp_df
    foursquare_df = pd.concat([foursquare_df, new_df])


In [13]:
# Drop unnecessary columns from dataframe
columns_to_drop_foursquare = ['short_name', 'plural_name', 'icon.prefix', 'icon.suffix']

foursquare_df = foursquare_df.drop(columns=columns_to_drop_foursquare)

# Rename columns
foursquare_df = foursquare_df.rename(columns={"id": "category_id", "name": "category_name", 'poi_location.address': 'poi_address'})

KeyError: "['short_name', 'plural_name', 'icon.prefix', 'icon.suffix'] not found in axis"

In [14]:
# Reorder columns
foursquare_df = foursquare_df[['poi_fsq_id', 'poi_name', 'category_id', 'category_name', 'poi_distance', 'poi_address', 'station_id']]

In [15]:
print(foursquare_df.shape, '\n')
print(foursquare_df.info())

(8069, 7) 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8069 entries, 0 to 12
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   poi_fsq_id     8069 non-null   object
 1   poi_name       8069 non-null   object
 2   category_id    8069 non-null   int64 
 3   category_name  8069 non-null   object
 4   poi_distance   8069 non-null   object
 5   poi_address    7713 non-null   object
 6   station_id     8069 non-null   object
dtypes: int64(1), object(6)
memory usage: 504.3+ KB
None


In [16]:
foursquare_df.head()

,poi_fsq_id,poi_name,category_id,category_name,poi_distance,poi_address,station_id
0,4e18b61d18a8166f738f1ed0,Mesón Pahi,13026,BBQ Joint,215,Calle Torroella de Montgrí 7,dedcd93fbe85671e73c79baf58bd7fed
1,4e18b61d18a8166f738f1ed0,Mesón Pahi,13338,Seafood Restaurant,215,Calle Torroella de Montgrí 7,dedcd93fbe85671e73c79baf58bd7fed
2,4e18b61d18a8166f738f1ed0,Mesón Pahi,13345,Spanish Restaurant,215,Calle Torroella de Montgrí 7,dedcd93fbe85671e73c79baf58bd7fed
3,590896ec5d891b372346f974,Cyclic Beer Farm,13029,Brewery,344,"Carrer de Coll, 37",dedcd93fbe85671e73c79baf58bd7fed
4,590896ec5d891b372346f974,Cyclic Beer Farm,13065,Restaurant,344,"Carrer de Coll, 37",dedcd93fbe85671e73c79baf58bd7fed


In [17]:
#save data to use in another notebook 
foursquare_df.to_csv('../data/bike_stations_barcelona_poi_foursquare.csv')

# Yelp

In [150]:
# imports
import requests
import os
import pandas as pd

In [151]:
API_KEY = os.environ.get("YELP_API_KEY")
# os.environ loads the dictionary with environment variables where os.environ.keys() are all variable names

In [159]:
API_KEY=os.environ.get("yelp_api_key1")

In [161]:
import requests

url = "https://api.yelp.com/v3/businesses/search?latitude=41.3874&longitude=2.1686&radius=1000&categories=bars%2C%20restaurant%2C%20pubs&sort_by=best_match&limit=20"

headers = {
    "accept": "application/json",
    "Authorization": "Bearer " + API_KEY
}

response = requests.get(url, headers=headers)
response_data = response.json()
print(response_data)

{'businesses': [{'id': '3YaqE9qZzE6gkQSFttWwPg', 'alias': 'viana-barcelona', 'name': 'Viana', 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/ef7goNmTtnO0PnmGpW3h0Q/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/viana-barcelona?adjust_creative=7zEqIiDnpKefJZuurfNouA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=7zEqIiDnpKefJZuurfNouA', 'review_count': 98, 'categories': [{'alias': 'spanish', 'title': 'Spanish'}, {'alias': 'bars', 'title': 'Bars'}], 'rating': 4.5, 'coordinates': {'latitude': 41.379814, 'longitude': 2.176219}, 'transactions': [], 'price': '€€€', 'location': {'address1': 'Carrer del Vidre, 7', 'address2': None, 'address3': '', 'city': 'Barcelona', 'zip_code': '08002', 'country': 'ES', 'state': 'B', 'display_address': ['Carrer del Vidre, 7', '08002 Barcelona', 'Spain']}, 'phone': '+34934638295', 'display_phone': '+34 934 63 82 95', 'distance': 1056.211323566262}, {'id': 'Cn1geHmwrvKu1qQISgQKgA', 'alias': 'sensi-barcelona', 'name':

In [162]:
response.json()

{'businesses': [{'id': '3YaqE9qZzE6gkQSFttWwPg',
   'alias': 'viana-barcelona',
   'name': 'Viana',
   'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/ef7goNmTtnO0PnmGpW3h0Q/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/viana-barcelona?adjust_creative=7zEqIiDnpKefJZuurfNouA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=7zEqIiDnpKefJZuurfNouA',
   'review_count': 98,
   'categories': [{'alias': 'spanish', 'title': 'Spanish'},
    {'alias': 'bars', 'title': 'Bars'}],
   'rating': 4.5,
   'coordinates': {'latitude': 41.379814, 'longitude': 2.176219},
   'transactions': [],
   'price': '€€€',
   'location': {'address1': 'Carrer del Vidre, 7',
    'address2': None,
    'address3': '',
    'city': 'Barcelona',
    'zip_code': '08002',
    'country': 'ES',
    'state': 'B',
    'display_address': ['Carrer del Vidre, 7', '08002 Barcelona', 'Spain']},
   'phone': '+34934638295',
   'display_phone': '+34 934 63 82 95',
   'distance': 1056.211323

In [163]:
print(response_data['businesses'])

[{'id': '3YaqE9qZzE6gkQSFttWwPg', 'alias': 'viana-barcelona', 'name': 'Viana', 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/ef7goNmTtnO0PnmGpW3h0Q/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/viana-barcelona?adjust_creative=7zEqIiDnpKefJZuurfNouA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=7zEqIiDnpKefJZuurfNouA', 'review_count': 98, 'categories': [{'alias': 'spanish', 'title': 'Spanish'}, {'alias': 'bars', 'title': 'Bars'}], 'rating': 4.5, 'coordinates': {'latitude': 41.379814, 'longitude': 2.176219}, 'transactions': [], 'price': '€€€', 'location': {'address1': 'Carrer del Vidre, 7', 'address2': None, 'address3': '', 'city': 'Barcelona', 'zip_code': '08002', 'country': 'ES', 'state': 'B', 'display_address': ['Carrer del Vidre, 7', '08002 Barcelona', 'Spain']}, 'phone': '+34934638295', 'display_phone': '+34 934 63 82 95', 'distance': 1056.211323566262}, {'id': 'Cn1geHmwrvKu1qQISgQKgA', 'alias': 'sensi-barcelona', 'name': 'Sensi', 'imag

In [164]:
response_data['businesses'][0]

{'id': '3YaqE9qZzE6gkQSFttWwPg',
 'alias': 'viana-barcelona',
 'name': 'Viana',
 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/ef7goNmTtnO0PnmGpW3h0Q/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/viana-barcelona?adjust_creative=7zEqIiDnpKefJZuurfNouA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=7zEqIiDnpKefJZuurfNouA',
 'review_count': 98,
 'categories': [{'alias': 'spanish', 'title': 'Spanish'},
  {'alias': 'bars', 'title': 'Bars'}],
 'rating': 4.5,
 'coordinates': {'latitude': 41.379814, 'longitude': 2.176219},
 'transactions': [],
 'price': '€€€',
 'location': {'address1': 'Carrer del Vidre, 7',
  'address2': None,
  'address3': '',
  'city': 'Barcelona',
  'zip_code': '08002',
  'country': 'ES',
  'state': 'B',
  'display_address': ['Carrer del Vidre, 7', '08002 Barcelona', 'Spain']},
 'phone': '+34934638295',
 'display_phone': '+34 934 63 82 95',
 'distance': 1056.211323566262}

In [165]:
# Load the df of the bike stations containing the latitude and longitude
bike_stations_load2 = pd.read_csv('../data/bike_stations_barcelona.csv', index_col=0)

In [166]:
# Create an empty dataframe
yelp_df = pd.DataFrame()

In [167]:
# Create list with desired categories from Yelp
dining_cat_yelp = ['restaurants', 'bars', 'pubs']

In [192]:
# Function to make an http request to yelp for each bike station
def get_poi_yelp(latitude, longitude, radius, API_KEY, categories):
    url = f'https://api.yelp.com/v3/businesses/search?latitude={latitude}&longitude={longitude}&radius={radius}&categories={categories}&sort_by=best_match'
    
    # Create dictionary for headers
    headers = {'Accept': 'application/json'}
    headers['Authorization'] = 'Bearer %s' % API_KEY
    response = requests.get(url, headers=headers)
    response_data = response.json()  

    return response_data['businesses']

In [193]:
# testing
res_2 = get_poi_yelp(41.3874, 2.1686,1000, API_KEY, categories=','.join(dining_cat_yelp))

In [194]:
# to explore the result of a single POI from the testing call
print(res_2)

[{'id': 'zrxsTCz5lt8q73Hx0GHLDw', 'alias': 'cerveseria-catalana-barcelona', 'name': 'Cerveseria Catalana', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/7UqXBGtaD7F1eovQZ3OtTw/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/cerveseria-catalana-barcelona?adjust_creative=7zEqIiDnpKefJZuurfNouA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=7zEqIiDnpKefJZuurfNouA', 'review_count': 1569, 'categories': [{'alias': 'spanish', 'title': 'Spanish'}], 'rating': 4.5, 'coordinates': {'latitude': 41.392393, 'longitude': 2.160993}, 'transactions': [], 'price': '€€', 'location': {'address1': 'Carrer de Mallorca, 236', 'address2': '', 'address3': None, 'city': 'Barcelona', 'zip_code': '08008', 'country': 'ES', 'state': 'B', 'display_address': ['Carrer de Mallorca, 236', '08008 Barcelona', 'Spain']}, 'phone': '+34932160368', 'display_phone': '+34 932 16 03 68', 'distance': 843.174081213008}, {'id': 'XhNHk-O5Uv2WO_9iFtP_gw', 'alias': 'ciutat-comtal-barcelona-3'

In [197]:
# to explore the result of a single POI from the testing call
print(res_2[0].keys(), '\n')
res_2['businesses'][0]

dict_keys(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count', 'categories', 'rating', 'coordinates', 'transactions', 'price', 'location', 'phone', 'display_phone', 'distance']) 



TypeError: list indices must be integers or slices, not str

In [198]:
print(response_data['businesses'])

[{'id': '3YaqE9qZzE6gkQSFttWwPg', 'alias': 'viana-barcelona', 'name': 'Viana', 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/ef7goNmTtnO0PnmGpW3h0Q/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/viana-barcelona?adjust_creative=7zEqIiDnpKefJZuurfNouA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=7zEqIiDnpKefJZuurfNouA', 'review_count': 98, 'categories': [{'alias': 'spanish', 'title': 'Spanish'}, {'alias': 'bars', 'title': 'Bars'}], 'rating': 4.5, 'coordinates': {'latitude': 41.379814, 'longitude': 2.176219}, 'transactions': [], 'price': '€€€', 'location': {'address1': 'Carrer del Vidre, 7', 'address2': None, 'address3': '', 'city': 'Barcelona', 'zip_code': '08002', 'country': 'ES', 'state': 'B', 'display_address': ['Carrer del Vidre, 7', '08002 Barcelona', 'Spain']}, 'phone': '+34934638295', 'display_phone': '+34 934 63 82 95', 'distance': 1056.211323566262}, {'id': 'Cn1geHmwrvKu1qQISgQKgA', 'alias': 'sensi-barcelona', 'name': 'Sensi', 'imag

In [199]:
# Set the maximum number of API calls
max_api_calls = 50

# Counter for the number of API calls made
api_calls_count = 0

# Loop through the bike_stations df to extract the lat/long/id of each row and make a new http request
for index, row in bike_stations_load2.iterrows():
    # Make the http request using row attributes
    poi_yelp = get_poi_yelp(row['latitude'], row['longitude'], 1000, API_KEY, categories=','.join(dining_cat_yelp))
    
    # Check if the maximum number of API calls has been reached
    if api_calls_count >= max_api_calls:
        break

    # Turn the response into a dataframe
    new_df = pd.json_normalize(poi_yelp)
    # Add a column with the corresponding bike station id
    new_df['station_id'] = row['id'] 
    # Add the new dataframe to yelp_df
    yelp_df = pd.concat([yelp_df, new_df]) 

    # Increment the API calls counter
    api_calls_count += 1


In [200]:
print(yelp_df.shape, '\n')
print(yelp_df.info())

(1203, 27) 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1203 entries, 0 to 19
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   businesses                211 non-null    object 
 1   total                     211 non-null    float64
 2   station_id                1203 non-null   object 
 3   id                        992 non-null    object 
 4   alias                     992 non-null    object 
 5   name                      992 non-null    object 
 6   image_url                 992 non-null    object 
 7   is_closed                 992 non-null    object 
 8   url                       992 non-null    object 
 9   review_count              992 non-null    float64
 10  categories                992 non-null    object 
 11  rating                    992 non-null    float64
 12  transactions              992 non-null    object 
 13  price                     805 non-null    object 
 1

In [201]:
# Drop unnecessary columns from dataframe
columns_to_drop_yelp = [
    'alias', 'image_url', 'is_closed', 'url', 'transactions', 'phone',
    'display_phone', 'coordinates.latitude',
    'coordinates.longitude', 'location.address1', 'location.address2',
    'location.address3', 'location.city', 'location.zip_code',
    'location.country', 'location.state','categories']

yelp_df = yelp_df.drop(columns=columns_to_drop_yelp)

# Rename columns
yelp_df = yelp_df.rename(columns={"id": "poi_id", "location.display_address": "full_address"})

In [202]:
print(yelp_df.shape, '\n')
print(yelp_df.info())

(1203, 10) 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1203 entries, 0 to 19
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   businesses    211 non-null    object 
 1   total         211 non-null    float64
 2   station_id    1203 non-null   object 
 3   poi_id        992 non-null    object 
 4   name          992 non-null    object 
 5   review_count  992 non-null    float64
 6   rating        992 non-null    float64
 7   price         805 non-null    object 
 8   distance      992 non-null    float64
 9   full_address  992 non-null    object 
dtypes: float64(4), object(6)
memory usage: 103.4+ KB
None


In [203]:
print(yelp_df.head())

                                          businesses   total  \
0  [{'id': 'J1vHLZvSRfkNpy2x8OuVaQ', 'alias': 'to...   124.0   
0  [{'id': 'uY7qDqn533Z0Rb3RMMBvmw', 'alias': 're...    95.0   
0  [{'id': '8GS5ZJFm93yuEwPFm8Vpiw', 'alias': 'el...    54.0   
0  [{'id': '3HiEACBcj1YY9SjspVZykQ', 'alias': 'ca...  1600.0   
0  [{'id': 'ioeSUAb2WZZpaq-IPJqWdg', 'alias': 'ti...   632.0   

                         station_id poi_id name  review_count  rating price  \
0  dedcd93fbe85671e73c79baf58bd7fed    NaN  NaN           NaN     NaN   NaN   
0  5c10c2dba1c5a8e8d9dcbb251fd8eab7    NaN  NaN           NaN     NaN   NaN   
0  f1aaf083aaa04498aad975bc45bdc089    NaN  NaN           NaN     NaN   NaN   
0  4a30020e6926a2ba39f88be0f6bba5cb    NaN  NaN           NaN     NaN   NaN   
0  e964297fbcd2fc96bb6632e5a530d0df    NaN  NaN           NaN     NaN   NaN   

   distance full_address  
0       NaN          NaN  
0       NaN          NaN  
0       NaN          NaN  
0       NaN          NaN  
0    

In [185]:
# Drop unnecessary columns from dataframe
columns_to_drop_yelp = ['businesses','total  ']

yelp_df = yelp_df.drop(columns=columns_to_drop_yelp)

In [186]:
yelp_df.head()

,businesses,total,station_id
0,"[{'id': 'J1vHLZvSRfkNpy2x8OuVaQ', 'alias': 'to...",124,dedcd93fbe85671e73c79baf58bd7fed
0,"[{'id': 'uY7qDqn533Z0Rb3RMMBvmw', 'alias': 're...",95,5c10c2dba1c5a8e8d9dcbb251fd8eab7
0,"[{'id': '8GS5ZJFm93yuEwPFm8Vpiw', 'alias': 'el...",54,f1aaf083aaa04498aad975bc45bdc089
0,"[{'id': '3HiEACBcj1YY9SjspVZykQ', 'alias': 'ca...",1600,4a30020e6926a2ba39f88be0f6bba5cb
0,"[{'id': 'ioeSUAb2WZZpaq-IPJqWdg', 'alias': 'ti...",632,e964297fbcd2fc96bb6632e5a530d0df


In [191]:
# Assuming your DataFrame is named df
businesses_column = yelp_df['businesses']

# Display the 'businesses' column
print(businesses_column)


0    [{'id': 'J1vHLZvSRfkNpy2x8OuVaQ', 'alias': 'to...
0    [{'id': 'uY7qDqn533Z0Rb3RMMBvmw', 'alias': 're...
0    [{'id': '8GS5ZJFm93yuEwPFm8Vpiw', 'alias': 'el...
0    [{'id': '3HiEACBcj1YY9SjspVZykQ', 'alias': 'ca...
0    [{'id': 'ioeSUAb2WZZpaq-IPJqWdg', 'alias': 'ti...
                           ...                        
0    [{'id': 'pdROjVIB39Z90DqWGZlX2A', 'alias': 're...
0    [{'id': 'PclBoxMV0j3vApdoVxOZkw', 'alias': 'bl...
0    [{'id': '2Eudmsaey5Aufe2r-D98HQ', 'alias': 'la...
0    [{'id': 'zrxsTCz5lt8q73Hx0GHLDw', 'alias': 'ce...
0    [{'id': '3HiEACBcj1YY9SjspVZykQ', 'alias': 'ca...
Name: businesses, Length: 211, dtype: object


In [188]:
yelp_df.head()['businesses']

0    [{'id': 'J1vHLZvSRfkNpy2x8OuVaQ', 'alias': 'to...
0    [{'id': 'uY7qDqn533Z0Rb3RMMBvmw', 'alias': 're...
0    [{'id': '8GS5ZJFm93yuEwPFm8Vpiw', 'alias': 'el...
0    [{'id': '3HiEACBcj1YY9SjspVZykQ', 'alias': 'ca...
0    [{'id': 'ioeSUAb2WZZpaq-IPJqWdg', 'alias': 'ti...
Name: businesses, dtype: object

In [176]:
yelp_df.head()

,businesses,total,region.center.longitude,region.center.latitude,station_id
0,"[{'id': 'J1vHLZvSRfkNpy2x8OuVaQ', 'alias': 'to...",124,2.185207,41.425364,dedcd93fbe85671e73c79baf58bd7fed
0,"[{'id': 'uY7qDqn533Z0Rb3RMMBvmw', 'alias': 're...",95,2.190246,41.430000,5c10c2dba1c5a8e8d9dcbb251fd8eab7
0,"[{'id': '8GS5ZJFm93yuEwPFm8Vpiw', 'alias': 'el...",54,2.190630,41.443365,f1aaf083aaa04498aad975bc45bdc089
0,"[{'id': '3HiEACBcj1YY9SjspVZykQ', 'alias': 'ca...",1600,2.170732,41.377532,4a30020e6926a2ba39f88be0f6bba5cb
0,"[{'id': 'ioeSUAb2WZZpaq-IPJqWdg', 'alias': 'ti...",632,2.152309,41.375116,e964297fbcd2fc96bb6632e5a530d0df


In [178]:
#save data to use in another notebook 
yelp_df.to_csv('../data/bike_stations_barcelona_poi_yelp1.csv')

# tried to get yelp code but API calls ran out

In [155]:
print('Foursquare results')
foursquare_df.head()

Foursquare results


,poi_fsq_id,poi_name,category_id,category_name,poi_distance,poi_address,station_id
0,4e18b61d18a8166f738f1ed0,Mesón Pahi,13026,BBQ Joint,215,Calle Torroella de Montgrí 7,dedcd93fbe85671e73c79baf58bd7fed
1,4e18b61d18a8166f738f1ed0,Mesón Pahi,13338,Seafood Restaurant,215,Calle Torroella de Montgrí 7,dedcd93fbe85671e73c79baf58bd7fed
2,4e18b61d18a8166f738f1ed0,Mesón Pahi,13345,Spanish Restaurant,215,Calle Torroella de Montgrí 7,dedcd93fbe85671e73c79baf58bd7fed
3,590896ec5d891b372346f974,Cyclic Beer Farm,13029,Brewery,344,"Carrer de Coll, 37",dedcd93fbe85671e73c79baf58bd7fed
4,590896ec5d891b372346f974,Cyclic Beer Farm,13065,Restaurant,344,"Carrer de Coll, 37",dedcd93fbe85671e73c79baf58bd7fed


In [179]:
print('Yelp results')
yelp_df.head()

Yelp results


,businesses,total,region.center.longitude,region.center.latitude,station_id
0,"[{'id': 'J1vHLZvSRfkNpy2x8OuVaQ', 'alias': 'to...",124,2.185207,41.425364,dedcd93fbe85671e73c79baf58bd7fed
0,"[{'id': 'uY7qDqn533Z0Rb3RMMBvmw', 'alias': 're...",95,2.190246,41.430000,5c10c2dba1c5a8e8d9dcbb251fd8eab7
0,"[{'id': '8GS5ZJFm93yuEwPFm8Vpiw', 'alias': 'el...",54,2.190630,41.443365,f1aaf083aaa04498aad975bc45bdc089
0,"[{'id': '3HiEACBcj1YY9SjspVZykQ', 'alias': 'ca...",1600,2.170732,41.377532,4a30020e6926a2ba39f88be0f6bba5cb
0,"[{'id': 'ioeSUAb2WZZpaq-IPJqWdg', 'alias': 'ti...",632,2.152309,41.375116,e964297fbcd2fc96bb6632e5a530d0df
